## Import libraries and dependencies


In [2]:
import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da
import optimiseprime.forecast as forecast
from optimiseprime.MCForecastTools import MCSimulation
import alpaca_trade_api

import datetime as dt


import pandas as pd
import numpy as np
import os
import yfinance as yf
import quantstats as qs
qs.extend_pandas()


from statsmodels.tsa.ar_model import AutoReg
from math import sqrt
from statsmodels.graphics.tsaplots import plot_pacf

import panel as pn
import panel.widgets as pnw
import pandas as pd

from matplotlib import pyplot as plt
import holoviews as hv
import hvplot.pandas
import plotly.express as px

# Additional
import matplotlib.pyplot as plt
import plotly.graph_objs as go    
import plotly.io as pio
pio.renderers.default ='colab'  
import seaborn as sns


%matplotlib inline

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [116]:
portfolio_choice = 0

while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio, type 2\n"
        )
                              )
    # Print error message if type(portfolio_choice) != int                        
    except ValueError:
        print('Error: Invalid response.\n')
        
    # Print error message if portfolio choice not 1 or 2
    if portfolio_choice not in [1, 2]:
        print('Error: Invalid response.\n')

# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        
        # Create a list of only the tickers
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
        
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")   


To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio, type 2
 1


Please enter the tickers of each cryptocurrency, followed by the number of units you hold:
Type 'done' when finished.


Ticker:  ETH
No. of units:  2
Ticker:  BTC
No. of units:  0.5
Ticker:  LTC
No. of units:  100
Ticker:  done


In [117]:
# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)

# Keep only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in list(portfolio_df.columns.levels[0])]

# Print portfolio data for visual confirmation
if portfolio_choice == 1:    
    # Calculate portfolio value of each cryptocurrency held
    for ticker in existing_portfolio:
        existing_portfolio[ticker].append(
            {'value': portfolio_df[ticker].iloc[-1, 3] * existing_portfolio[ticker][0]['units']})
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    portfolio_value = 0
    for ticker, units in existing_portfolio.items():
        value = existing_portfolio[ticker][1]['value']
        print(f"Value of {existing_portfolio[ticker][0]['units']} {ticker}: ${value:.2f}")
        portfolio_value += value
    print(f"\nTotal portfolio value: ${portfolio_value:.2f}\n")       
    
elif portfolio_choice == 2:
    print(f"--------------------------")                      
    print(f"Hypothetical Portfolio:")
    print(f"{[ticker.replace('-USD', '') for ticker in ticker_list]}")                   
    print(f"Investment amount:")
    print(f"${portfolio_value:.2f}\n")

print(
    f"NOTE:\n"
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")




--------------------------
Existing Portfolio:
Value of 2.0 ETH-USD: $6611.89
Value of 0.5 BTC-USD: $21446.08
Value of 100.0 LTC-USD: $14826.61

Total portfolio value: $42884.58

NOTE:
To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.
While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.
Earliest date for which price data is available for all cryptocurrencies in your portfolio: 2019-01-17
--------------------------


## Data Analyses

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [118]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adj_sortino': adjusted_sortino,
    'gain_pain': gain_pain_ratio,
    }
)


# Calculate  weights according to the proportion scores for each risk-reward metric
weights = da.calculate_weights(ratios_df)

# If user has selected to analyse an existing portfolio, add user's portfolio weights as well.
if portfolio_choice == 1 :
    user_weights = []
    for ticker, stats in existing_portfolio.items():
        holding_value = stats[1]['value']
        user_weights.append(holding_value / portfolio_value)

    weights = pd.concat(
        [
            da.calculate_weights(ratios_df), pd.DataFrame(user_weights, columns = ['user_portfolio'], index = ticker_list),
        ],
        join = 'inner',
        axis = 'columns'
    )


In [121]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order

for column in ratios_df:
    if column == 'sharpe':
        sharpe_portfolio = da.sharpe_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'sortino':
        sortino_portfolio = da.sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'adj_sortino':
        adj_sortino_portfolio = da.adj_sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'gain_pain':
        gain_pain_portfolio = da.gain_pain_portfolio(ratios_df, weights, portfolio_value)

# If user portfolio_choice == 1 (i.e. existing portfolio), print user portfolio statistics as well
if portfolio_choice == 1:
    user_portfolio = da.user_portfolio(weights, portfolio_value)


Portfolio allocation recommendations
Based on historical returns from 2019-01-17 to 2022-01-15
Total portfolio value: $42884.58

Metric: Sharpe Ratio
--------------
ETH-USD                            1.65 
Recommended % of total portfolio   39.68%
Recommended value allocation       $17016.61

BTC-USD                            1.49 
Recommended % of total portfolio   35.82%
Recommended value allocation       $15360.52

LTC-USD                            1.02 
Recommended % of total portfolio   24.50%
Recommended value allocation       $10507.45


Metric: Sortino Ratio
--------------
ETH-USD                            2.45 
Recommended % of total portfolio   39.59%
Recommended value allocation       $16978.21

BTC-USD                            2.23 
Recommended % of total portfolio   36.02%
Recommended value allocation       $15448.10

LTC-USD                            1.51 
Recommended % of total portfolio   24.39%
Recommended value allocation       $10458.27


Metric: Adjusted Sorti

## Autoregressive Modelling and Forecasting

In [123]:
ticker_close_names = []
for ticker in list(weights.index):
    ticker_close_names.append((ticker, 'close'))    
    
portfolio_close_df = portfolio_df[ticker_close_names]

portfolio_returns_list = []
for column_name in weights.columns:
    weights_list = weights[column_name].tolist()
    portfolio_returns = portfolio_close_df.dot(weights_list)
    portfolio_returns = portfolio_returns.rename(column_name)
    portfolio_returns_list.append(portfolio_returns)


In [124]:

true_false = pn.widgets.Select(name='View whole graph', options=[True, False])

    
interact = pn.interact(
    forecast.forecast_all_portfolios, 
    portfolio_returns_list = [portfolio_returns_list[0:5]], 
    window = range(365), 
    foward_looking = range(365), 
    full_picture = true_false
)

pn.Column(interact[1], interact[0])

Column
    [0] Row
        [0] HoloViews(NdOverlay, name='interactive75873')
    [1] Column
        [0] Select(name='portfolio_returns_list', options=[[Date
2019-01-17     1374...], value=[Date
2019-01-17     1374....])
        [1] DiscreteSlider(formatter='%d', name='window', options=[0, 1, 2, 3, 4, ...], value=0)
        [2] DiscreteSlider(formatter='%d', name='foward_looking', options=[0, 1, 2, 3, 4, ...], value=0)
        [3] Select(name='View whole graph', options=[True, False], value=True)

## Monte Carlo Simulation Forecasting

In [125]:
#Set number of simulations 
num_sims = 500
num_trading_days = 365 * (int(input("Key in number of years to forecast:")))

MC_sharpe = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sharpe'].tolist(),
    num_trading_days = num_trading_days
)

MC_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_adj_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['adj_sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_gain_pain = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['gain_pain'].tolist(),
    num_trading_days = num_trading_days
)


simulations_list = [MC_sharpe, MC_sortino, MC_adj_sortino, MC_gain_pain]

if portfolio_choice == 1:
    MC_user = MCSimulation(
        portfolio_data = portfolio_df,
        num_simulation = num_sims,
        weights = weights['user_portfolio'].tolist(),
        num_trading_days = num_trading_days
    )
    simulations_list.append(MC_user)

for simulation in simulations_list:
    simulation.calc_cumulative_return()

    


Key in number of years to forecast: 1


Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


D:\Schoolwork\03 Monash FinTech Bootcamp 21-22\04_project\Group1-Project1\optimiseprime\MCForecastTools.py:117: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

In [129]:
# Compute summary statistics from the simulated daily returns, for each simulation

all_cumulative_pnl = []
for simulation in simulations_list:
    simulated_returns = pd.DataFrame(
        {
            "mean": list(simulation.simulated_return.mean(axis=1)),
            "median": list(simulation.simulated_return.median(axis=1)),            
            "min": list(simulation.simulated_return.min(axis=1)),
            "max": list(simulation.simulated_return.max(axis=1))
        }
    )
    all_cumulative_pnl.append(simulated_returns)

# Concatenate cumulative pnls predicted by MC simulations for each strategy
all_mean_cumulative_pnl = pd.concat([df['mean'] for df in all_cumulative_pnl], join = 'inner', axis = 'columns') 
all_mean_cumulative_pnl = all_mean_cumulative_pnl * portfolio_value
#for df in all_cumulative_pnl:
    #data_mean = pd.DataFrame(df['mean'])
    #all_mean_cumulative_pnl = pd.concat([data_mean], axis = 'columns', join = 'inner')
all_mean_cumulative_pnl.columns = weights.columns.tolist()



In [130]:
px.line(
    all_mean_cumulative_pnl,
    title = f"Simulated mean portfolio values over the next {num_trading_days / 365} years by R-R strategies",
    labels = {
        "value": "Portfolio Value ($)",
        "index": "Number of Trading Days"
        
    }
)

